In [4]:
from visualization_fct import *
# from bokeh.plotting import output_file, show, save

from bokeh.resources import CDN
from bokeh.embed import file_html
from bokeh.plotting import show


from pomegranate import GeneralMixtureModel
from pomegranate import MultivariateGaussianDistribution

import matplotlib.pyplot as plt  # , mpld3

without_CA = False

data = pd.read_csv("../asm_data/asm_data_for_ml.txt", sep='\t')
del data['MJD']
del data['error']
del data['errorA']
del data['errorB']
del data['errorC']
data['rateCA'] = data.rateC / data.rateA
data_thr = mask(data, 'orbit')  # rm too large values except for 'orbit'
data_thr = data

np.random.seed(0)

if without_CA:
    X = np.c_[data_thr.orbit, data_thr.rate, data_thr.rateA, data_thr.rateB,
              data_thr.rateC]
else:
    X = np.c_[data_thr.orbit, data_thr.rate, data_thr.rateA, data_thr.rateB,
              data_thr.rateC, data_thr.rateCA]


gmm = GeneralMixtureModel(MultivariateGaussianDistribution, n_components=3)
gmm.fit(X)

preds = gmm.predict(X)
probs = gmm.predict_proba(X)

data_thr['preds'] = pd.Series(preds).astype("category")

color_key = ["red", "blue", "yellow", "grey", "black", "purple", "pink",
             "brown", "green", "orange"]  # Spectral9
color_key = color_key[:len(set(preds))+1]

covs = np.array([np.array(gmm.distributions[m].parameters[1]) for m in range(len(gmm.distributions))])
means = np.array([np.array(gmm.distributions[m].parameters[0]) for m in range(len(gmm.distributions))])


IOError: File ../asm_data/asm_data_for_ml.txt does not exist

In [ ]:
%matplotlib notebook
output_notebook()

In [ ]:
p = plot_probs_bokeh_linked_brushing(data_thr, covs=covs_xy, means=means_xy)
show(p)